In [ ]:
from pathlib import Path
import os
import json

from mozaik.tools.distribution_parametrization import load_parameters

PROJECT_ROOT = Path('/home/haman/layers56/')


In [57]:
def flatten_dict(d, parent_key='', sep='.'):
    """
    Flatten a nested dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def unflatten_dict(d, sep='.'):
    """
    Unflatten a flattened dictionary.
    """
    result = {}
    for key, value in d.items():
        parts = key.split(sep)
        current = result
        for part in parts[:-1]:
            if part not in current:
                current[part] = {}
            current = current[part]
        current[parts[-1]] = value
    return result

def compare_parameters(param1:dict, param2:dict, ignore_keys=None):
    """
    Compare two ParameterSet objects and return differences.
    """
    if ignore_keys is None:
        ignore_keys = set()
    flat1 = flatten_dict(param1)
    flat2 = flatten_dict(param2)
    
    differences = {}
    for key in set(flat1.keys()).union(flat2.keys()):
        if any(ignored in key for ignored in ignore_keys):
            continue
        if flat1.get(key) != flat2.get(key):
            differences[key] = (flat1.get(key), flat2.get(key))
    
    return differences

def load_parameters_with_references(param_path):
    """ Load parameters while keeping values with references as string
    """

    param_path = (Path.cwd() / param_path).parent
    param_name = param_path.name

    tmp_path = param_path / "tmp"
    tmp_path.mkdir(parents=False, exist_ok=False)

    for file_name in param_path.glob("*"):
        if file_name.is_dir():
            # print(f"Skipping directory: {file_name}")
            continue
        # print(f"Processing file: {file_name}")
        with open(file_name, 'r') as src_file, open(tmp_path / file_name.name, 'w') as dest_file:
            string_flag = False
            for line in src_file:
                line = line.replace('"',"'")
                if 'url(' in line:
                    line = line.replace(param_name, f'{param_name}/tmp')

                if 'ref(' in line or string_flag:
                    if ":" in line:
                        line = line. replace(':', ':"')
                        string_flag = True

                    if line.rstrip().endswith(','):
                        line = line.replace(',', '",')  # add string end "
                        string_flag = False
                    elif "\\" not in line and (line.count('(') == line.count(')')):
                        line = line.replace("\n", '",\n')
                        string_flag = False
                dest_file.write(line)

    str_param = load_parameters(f"{param_name}/tmp/defaults",{}).to_dict()
    str_param = flatten_dict(str_param)
    for key, value in str_param.items():
        if isinstance(value, str):
            str_param[key] = value.strip()
    str_param = unflatten_dict(str_param)

    for tmp_file in tmp_path.glob("*"):
        # print(f"Removing temporary file: {tmp_file}")
        tmp_file.unlink()
    # print(f"Removing temporary directory: {tmp_path}")
    tmp_path.rmdir() 

    return str_param

def compare_model_parameters(model_path, ignore_keys):
    print(f"Comparing parameters for {model_path}\n")
    os.chdir(model_path)

    print("Comparing LOADED parameters:")
    param = load_parameters("param/defaults",{}).to_dict()
    param_spont = load_parameters("param_spont/defaults", {}).to_dict()
    try:
        param_optim = load_parameters("param_optim/defaults", {}).to_dict()
    except (NameError, FileNotFoundError):
        print("No 'param_optim' found, skipping optimization parameters.")
        param_optim = None
    print('Differences between "param"       and  "param_spont": ', compare_parameters(param, param_spont, ignore_keys))
    if param_optim:
        print('Differences between "param"       and  "param_optim": ', compare_parameters(param, param_optim, ignore_keys))
        print('Differences between "param_spont" and  "param_optim": ', compare_parameters(param_spont, param_optim, ignore_keys))
    print("="*80)

    print('Checking recorders:')

    param_list = [param, param_spont]
    recorder_list = [recs, recs_spont]
    if param_optim:
        param_list.append(param_optim)
        recorder_list.append(recs_optim)
    for p, r in zip(param_list, recorder_list):
        for sheet in p['sheets'].keys():
            s = sheet.split('_')
            if s[1] == 'cortex':
                assert p['sheets'][sheet]['params']['recorders'] == r[f'{s[0]}_{s[-1]}_rec']
            elif s[1] == 'lgn':
                assert p['sheets'][sheet]['params']['recorders'] == r[f'{s[-1]}_rec']
            else:
                print(f"Unknown sheet type: {sheet}")
    print('All recorders match the predefined ones.')
    print("="*80)

    print("Comparing parameters with REFS:")

    param = load_parameters_with_references("param/defaults")
    param_spont = load_parameters_with_references("param_spont/defaults")

    try:
        param_optim = load_parameters_with_references("param_optim/defaults")
    except (NameError, FileNotFoundError):
        print("No 'param_optim' found, skipping optimization parameters.")
        param_optim = None
    print('Differences between "param"       and  "param_spont": ', compare_parameters(param, param_spont, ignore_keys))
    if param_optim:
        print('Differences between "param"       and  "param_optim": ', compare_parameters(param, param_optim, ignore_keys))
        print('Differences between "param_spont" and  "param_optim": ', compare_parameters(param_spont, param_optim, ignore_keys))

In [3]:
# predefined recorders
recs = {
    "l4_exc_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes","v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 200.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 10.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "3" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 600.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "4" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 2000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 100.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "l4_inh_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes","v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 200.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "3" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 600.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "l23_exc_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes","v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 200.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 10.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "3" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 600.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "4" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 2000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 100.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "l23_inh_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes","v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 200.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "3" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 600.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "lgn_rec": {
        "1" : {
            'component' :  'mozaik.sheets.population_selector.RCRandomN',
            'variables' : ("spikes","V_m","g_ex" , "g_in"),
            'params' :  {
                'num_of_cells' : 5,
            }
        },
    }
}

recs_spont = {
    "l4_exc_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 2000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 100.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "4" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 3000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "l4_inh_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 2000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 100.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "4" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 3000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "l23_exc_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 2000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 100.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "4" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 3000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "l23_inh_rec": {
        "2" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("v","gsyn_exc" , "gsyn_inh"),
            'params' :  {
                'size': 2000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 100.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
        "4" : {
            'component' :  'mozaik.sheets.population_selector.RCGrid',
            'variables' : ("spikes"),
            'params' :  {
                'size': 3000.0,  # the size of the grid (it is assumed to be square) - it has to be multiple of spacing (micro meters)
                'spacing' : 20.0, #the space between two electrodes (micro meters)
                'offset_x' : 0.0, # the x axis offset from the center of the sheet (micro meters)
                'offset_y' : 0.0, # the y axis offset from the center of the sheet (micro meters)
            }
        },
    },
    "lgn_rec": {
        "1" : {
            'component' :  'mozaik.sheets.population_selector.RCRandomN',
            'variables' : ("spikes","V_m","g_ex" , "g_in"),
            'params' :  {
                'num_of_cells' : 5,
            }
        },
    }
}

recs_optim = {
    "l4_exc_rec": {
        "1" : {
            'component':  'mozaik.sheets.population_selector.RCAll',
            'variables': ("spikes"),
            'params': {
            }
        }
    },
    "l4_inh_rec": {
        "1" : {
            'component':  'mozaik.sheets.population_selector.RCAll',
            'variables': ("spikes"),
            'params': {
            }
        }
    },
    "l23_exc_rec": {
        "1" : {
            'component':  'mozaik.sheets.population_selector.RCAll',
            'variables': ("spikes"),
            'params': {
            }
        }
    },
    "l23_inh_rec": {
        "1" : {
            'component':  'mozaik.sheets.population_selector.RCAll',
            'variables': ("spikes"),
            'params': {
            }
        }
    },
    "lgn_rec": {
        "1" : {
            'component' :  'mozaik.sheets.population_selector.RCRandomN',
            'variables' : ("spikes"),
            'params' :  {
                'num_of_cells' : 5,
            }
        }
    }
}

In [4]:
# Pick model

# model_name = 'LSV1M_split_full'

# model_name = 'LSV1M_split_ee'

model_name = 'LSV1M_optimization'

# model_name = 'LSV1M_tsodyks2'

# model_name = 'LSV1M_split_remy'

# model_name = 'LSV1M_refs-update'

# model_name = 'LSV1M_infragranular'


In [ ]:
ignore_keys = {"recorders"}

# model_name = 'LSV1M_split_full'
# model_name = 'LSV1M_split_ee'
# model_name = 'LSV1M_optimization'
# model_name = 'LSV1M_tsodyks2'
# model_name = 'LSV1M_split_remy'
# model_name = 'LSV1M_refs-update'
# model_name = 'LSV1M_infragranular'

model_list = [
    'LSV1M_split_full',
    'LSV1M_split_ee',
    'LSV1M_optimization',
    'LSV1M_tsodyks2',
    'LSV1M_remy',
    'LSV1M_refs-update',
    'LSV1M_infragranular',
]

for model_name in model_list:
    model_path = Path(f'/home/haman/layers56/{model_name}/')
    compare_model_parameters(model_path, ignore_keys)
    print ("\n", "="*80, "\n") 



Comparing parameters for /home/haman/layers56/LSV1M_split_full

Comparing LOADED parameters:
Differences between "param"       and  "param_spont":  {}
Differences between "param"       and  "param_optim":  {}
Differences between "param_spont" and  "param_optim":  {}
Checking recorders:
All recorders match the predefined ones.
Comparing parameters with REFS:
Differences between "param"       and  "param_spont":  {}
Differences between "param"       and  "param_optim":  {}
Differences between "param_spont" and  "param_optim":  {}


Comparing parameters for /home/haman/layers56/LSV1M_split_ee

Comparing LOADED parameters:
Differences between "param"       and  "param_spont":  {}
Differences between "param"       and  "param_optim":  {}
Differences between "param_spont" and  "param_optim":  {}
Checking recorders:
All recorders match the predefined ones.
Comparing parameters with REFS:
Differences between "param"       and  "param_spont":  {}
Differences between "param"       and  "param_op

In [43]:
param = load_parameters("param/defaults",{})
param.to_dict()

{'input_space_type': 'mozaik.space.VisualSpace',
 'input_space': {'update_interval': 7.0, 'background_luminance': 50.0},
 'visual_field': {'centre': (0.0, 0.0), 'size': (11.0, 11.0)},
 'sheets': {'l4_cortex_exc': {'component': 'mozaik.sheets.vision.VisualCorticalUniformSheet',
   'params': {'name': 'V1_Exc_L4',
    'sx': 5000.0,
    'sy': 5000.0,
    'density': 2000.0,
    'mpi_safe': False,
    'magnification_factor': 1000.0,
    'cell': {'model': 'EIF_cond_exp_isfa_ista',
     'native_nest': False,
     'params': {'v_rest': -80,
      'v_reset': -60.0,
      'tau_refrac': 2.0,
      'tau_m': 8.0,
      'cm': 0.032,
      'e_rev_E': 0.0,
      'e_rev_I': -80.0,
      'tau_syn_E': 1.5,
      'tau_syn_I': 4.2,
      'a': -0.8,
      'b': 0.08,
      'delta_T': 0.8,
      'tau_w': 1.0,
      'v_thresh': -57.0},
     'initial_values': {'v': {'class_name': 'PyNNDistribution',
       'params': {'name': 'uniform', 'low': -70, 'high': -51}}}},
    'artificial_stimulators': {},
    'recording_

In [ ]:
# Part II: Compare parameters of two models

model_name1 = 'LSV1M_optimization'
model_name2 = 'LSV1M_refs-update'

expected_differences = {'density', 'tau_psc'}
# expected_differences = {"L23ExcL23InhConnectionLong", "L23ExcL23InhConnectionShort"}#, "L23ExcL23InhConnection"}

################################################################################

print("Comparing LOADED parameters:")

model_path1 = Path(f'/home/haman/layers56/{model_name1}/')
os.chdir(model_path1)
param1 = load_parameters("param/defaults",{})

model_path2 = Path(f'/home/haman/layers56/{model_name2}/')
os.chdir(model_path2)
param2 = load_parameters("param/defaults",{})

print(f'Differences between models: "{model_name1}" and "{model_name2}": ')
for key, (value1, value2) in compare_parameters(param1, param2).items():
    if any(difference in key for difference in expected_differences):
        continue
    else:
        print(key, ":", value1, value2)



Comparing LOADED parameters:
Differences between models: "LSV1M_optimization" and "LSV1M_refs-update": 
sheets.l23_cortex_exc.L23ExcL4ExcConnection.short_term_plasticity.tau_rec : 30 20
sheets.l23_cortex_exc.L23ExcL4InhConnection.short_term_plasticity.tau_rec : 30 20


In [ ]:
# Part III: pick param folder and make copy of it with changed recorders
model_name = 'LSV1M_split_full'
parent_param = 'param'

param_dirs = ['param', 'param_spont', 'param_optim']
file_list = [
    "defaults",
    "l4_cortex_exc",
    "l4_cortex_inh",
    "l23_cortex_exc",
    "l23_cortex_inh",
    "SpatioTemporalFilterRetinaLGN_defaults"
]

recorders_list = [
    "l4_exc_rec",
    "l4_inh_rec",
    "l23_exc_rec",
    "l23_inh_rec",
    "lgn_rec",
]

for updated_param in param_dirs:
    if updated_param == parent_param:
        continue
    # copy the files
    # change recorders link
    for file_name in file_list:
        src_file = Path(f'/home/haman/layers56/{model_name}/{parent_param}/{file_name}')
        dest_file = Path(f'/home/haman/layers56/{model_name}/{updated_param}/{file_name}')
        if src_file.exists():
            dest_file.parent.mkdir(parents=True, exist_ok=True)
            with open(src_file, 'r') as src, open(dest_file, 'w') as dest:
                for line in src:
                    if 'url(' in line:
                        line = line.replace(parent_param, updated_param)
                    dest.write(line)
            print(f"Copied {src_file} to {dest_file}")
    # check recorders






In [ ]:
file_list = [
    "defaults",
    "l4_cortex_exc",
    "l4_cortex_inh",
    "l23_cortex_exc",
    "l23_cortex_inh",
    "SpatioTemporalFilterRetinaLGN_defaults"
]

recorders_list = [
    "l4_exc_rec",
    "l4_inh_rec",
    "l23_exc_rec",
    "l23_inh_rec",
    "lgn_rec",
]

In [ ]:

param_name = "param"
model_name = 'LSV1M_split_full'
os.chdir(Path(f'/home/haman/layers56/{model_name}'))
str_param = load_parameters_with_references(f'{param_name}/defaults')



Skipping directory: /home/haman/layers56/LSV1M_split_full/param/tmp
Processing file: /home/haman/layers56/LSV1M_split_full/param/l23_exc_rec
Processing file: /home/haman/layers56/LSV1M_split_full/param/l4_exc_rec
Processing file: /home/haman/layers56/LSV1M_split_full/param/l4_inh_rec
Processing file: /home/haman/layers56/LSV1M_split_full/param/l23_inh_rec
Processing file: /home/haman/layers56/LSV1M_split_full/param/l23_cortex_exc
Processing file: /home/haman/layers56/LSV1M_split_full/param/defaults
Processing file: /home/haman/layers56/LSV1M_split_full/param/lgn_rec
Processing file: /home/haman/layers56/LSV1M_split_full/param/SpatioTemporalFilterRetinaLGN_defaults
Processing file: /home/haman/layers56/LSV1M_split_full/param/l4_cortex_inh
Processing file: /home/haman/layers56/LSV1M_split_full/param/l4_cortex_exc
Processing file: /home/haman/layers56/LSV1M_split_full/param/l23_cortex_inh
Removing temporary file: /home/haman/layers56/LSV1M_split_full/param/tmp/SpatioTemporalFilterRetinaLG

In [9]:
param

{'input_space_type': 'mozaik.space.VisualSpace',
 'input_space': {'update_interval': 7.0, 'background_luminance': 50.0},
 'visual_field': {'centre': (0.0, 0.0), 'size': (11.0, 11.0)},
 'sheets': {'l4_cortex_exc': {'component': 'mozaik.sheets.vision.VisualCorticalUniformSheet',
   'params': {'name': 'V1_Exc_L4',
    'sx': 5000.0,
    'sy': 5000.0,
    'density': 2000.0,
    'mpi_safe': False,
    'magnification_factor': 1000.0,
    'cell': {'model': 'EIF_cond_exp_isfa_ista',
     'native_nest': False,
     'params': {'v_rest': -80,
      'v_reset': -60.0,
      'tau_refrac': 2.0,
      'tau_m': 8.0,
      'cm': 0.032,
      'e_rev_E': 0.0,
      'e_rev_I': -80.0,
      'tau_syn_E': 1.5,
      'tau_syn_I': 4.2,
      'a': -0.8,
      'b': 0.08,
      'delta_T': 0.8,
      'tau_w': 1.0,
      'v_thresh': -57.0},
     'initial_values': {'v': <mozaik.tools.distribution_parametrization.PyNNDistribution at 0x7a3034574e60>}},
    'artificial_stimulators': {},
    'recording_interval': 1.0,
   

In [19]:
str_param

{'input_space_type': 'mozaik.space.VisualSpace',
 'input_space': {'update_interval': 7.0, 'background_luminance': 50.0},
 'visual_field': {'centre': (0.0, 0.0), 'size': (11.0, 11.0)},
 'sheets': {'l4_cortex_exc': {'component': 'mozaik.sheets.vision.VisualCorticalUniformSheet',
   'params': {'name': 'V1_Exc_L4',
    'sx': 5000.0,
    'sy': "ref('sheets.l4_cortex_exc.params.sx')",
    'density': " 20000.0 * ref('density_frac')",
    'mpi_safe': False,
    'magnification_factor': 1000.0,
    'cell': {'model': 'EIF_cond_exp_isfa_ista',
     'native_nest': False,
     'params': {'v_rest': -80,
      'v_reset': -60.0,
      'tau_refrac': 2.0,
      'tau_m': 8.0,
      'cm': 0.032,
      'e_rev_E': 0.0,
      'e_rev_I': -80.0,
      'tau_syn_E': 1.5,
      'tau_syn_I': 4.2,
      'a': -0.8,
      'b': 0.08,
      'delta_T': 0.8,
      'tau_w': 1.0,
      'v_thresh': -57.0},
     'initial_values': {'v': <mozaik.tools.distribution_parametrization.PyNNDistribution at 0x7a30345746e0>}},
    'arti